- Efficient Yelp API Calls
- Oscar Ochoa
-4/30/2022

In [2]:
import pandas as pd
import json
import os
import time, math
from tqdm.notebook import tqdm_notebook
from yelpapi import YelpAPI

# Accessing API

In [3]:
with open("c:/Users/oscar/.secret/yelp_api.json") as f:
    login = json.load(f)
    
yelp_api = YelpAPI(login["api_key"], timeout_s=5.0)
yelp_api

# User Defined Functions

## Check if file exist

In [4]:
def file_exist(file):
    # checks if file exist
    file_exist = os.path.isfile(file)
    
    # if the file doesn't exist
    if file_exist == False:
        folder = os.path.dirname(file) # gets the folder name
        
        # if json included a folder
        if len(folder) > 0:
            # creates the folder
            os.makedirs(folder, exist_ok=True)
        
        # Informs user that folder is being created
        print(f"[i]{file} not found. Saving empty list to file,")
    
        with open(file, "w") as f:
            json.dump([], f)
    
    else:
        print(f"[i] {file} already exist!")
        

## Offset for pagination

In [5]:
def offset(file):
    with open(file, "r") as f:
        previous_results = json.load(f)
        
    n_results = len(previous_results)
    print(f"- {n_results} previous results found.")
    
    return n_results

## Obtain previous results

In [6]:
def get_previous_results(file):
    with open(file, "r") as f:
        previous_results = json.load(f)
    return previous_results

## Add results to json file

In [7]:
def add_results(old_results, new_results, file):
    old_results.extend(new_results)
    with open(file, "w") as f:
        json.dump(old_results, f)

## API Call

In [8]:
def call_api(file, search, pages):
    for i in tqdm_notebook(range(1, pages+1)):
        time.sleep(0.2)
    
        n_results = offset(file=file)
    
        results = yelp_api.search_query(location=LOCATION,
                                   term=TERM,
                                   offset=n_results)
    
        previous_results = get_previous_results(file=file)
        add_results(previous_results, results[search], file)

## Get number of pages

In [9]:
def get_n_pages(query, total, per_page):
    total_results = query[total]
    results_per_page = len(query[per_page])
    n_pages = math.ceil((total_results - n_results) / results_per_page)
    
    return n_pages

# Extracting data from Yelp API

In [10]:
LOCATION = "Bakersfield, CA, 93313"
TERM = "Sushi"

In [11]:
JSON_FILE = "Data/results_in_progress_Sushi.json"
JSON_FILE

'Data/results_in_progress_Sushi.json'

In [12]:
# checking if file exist
file_exist(file=JSON_FILE)

[i] Data/results_in_progress_Sushi.json already exist!


In [13]:
# check to see if any data is inside file
n_results = offset(file=JSON_FILE)

- 81 previous results found.


In [14]:
# calling api
results = yelp_api.search_query(location=LOCATION,
                               term=TERM)

In [15]:
# obtaining our keys 
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [16]:
# checking how much data our api call obtained
results["total"]

81

In [17]:
# getting number of pages needed for our api call
n_pages = get_n_pages(query=results, total="total", per_page="businesses")
n_pages

0

In [18]:
call_api(file=JSON_FILE, search="businesses", pages=n_pages)

0it [00:00, ?it/s]

## Converting json into pandas Data Frame

In [18]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,sPhWfZXoJ3AhYqeR9HodPQ,sushi-n-grill-bakersfield,Sushi N Grill,https://s3-media3.fl.yelpcdn.com/bphoto/S36gaW...,False,https://www.yelp.com/biz/sushi-n-grill-bakersf...,199,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 35.30989, 'longitude': -119.0401}","[pickup, delivery]",$$,"{'address1': '3401 Pacheco Rd', 'address2': 'S...",+16618344227,(661) 834-4227,15655.082715
1,SiEipYoC1-c_zxmx4sFk6g,umi-sushi-and-sake-bakersfield-2,Umi Sushi & Sake,https://s3-media2.fl.yelpcdn.com/bphoto/yPl_sG...,False,https://www.yelp.com/biz/umi-sushi-and-sake-ba...,326,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 35.323542, 'longitude': -119.1265...","[pickup, delivery]",$$,"{'address1': '3300 Buena Vista Rd', 'address2'...",+16615648020,(661) 564-8020,18694.477984
2,-xPJ4s-NVzM-Ncn0q6q2vA,jin-sushi-downtown-bakersfield,JIN Sushi- Downtown,https://s3-media3.fl.yelpcdn.com/bphoto/HrbE00...,False,https://www.yelp.com/biz/jin-sushi-downtown-ba...,624,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 35.3765182, 'longitude': -119.020...","[pickup, delivery]",$$,"{'address1': '1600 19th St', 'address2': '', '...",+16614474622,(661) 447-4622,23167.466760
3,aNBmx-Lm4hkqwiXDymVEYA,love-sushi-bakersfield-3,Love Sushi,https://s3-media1.fl.yelpcdn.com/bphoto/uommoc...,False,https://www.yelp.com/biz/love-sushi-bakersfiel...,188,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",3.5,"{'latitude': 35.2945541, 'longitude': -119.075...","[pickup, delivery]",$$,"{'address1': '6515 Panama Ln', 'address2': 'St...",+16613988800,(661) 398-8800,14223.153744
4,WuTohPMzH-z5uj39ldHPTQ,jin-sushi-brimhall-and-allen-bakersfield,JIN Sushi- Brimhall & Allen,https://s3-media3.fl.yelpcdn.com/bphoto/a8wTBn...,False,https://www.yelp.com/biz/jin-sushi-brimhall-an...,90,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 35.369264, 'longitude': -119.146288}","[pickup, delivery]",$$,"{'address1': '12900 Brimhall Rd', 'address2': ...",+16614593400,(661) 459-3400,24075.191853


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
76,WnxL_Ip3zzlXJKZadtmSzA,chinatown-buffet-bakersfield-2,Chinatown Buffet,https://s3-media1.fl.yelpcdn.com/bphoto/lY4jdq...,False,https://www.yelp.com/biz/chinatown-buffet-bake...,94,"[{'alias': 'buffets', 'title': 'Buffets'}, {'a...",2.5,"{'latitude': 35.36782, 'longitude': -119.03946}",[],$$,"{'address1': '1021 Oak St', 'address2': '', 'a...",+16613243189,(661) 324-3189,22094.538082
77,9HAkEFLc9rK211uK3MOuow,vons-bakersfield-8,Vons,https://s3-media3.fl.yelpcdn.com/bphoto/93-iQu...,False,https://www.yelp.com/biz/vons-bakersfield-8?ad...,61,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.0,"{'latitude': 35.3416605, 'longitude': -119.102...",[],$$,"{'address1': '9000 Ming Ave', 'address2': '', ...",+16616630595,(661) 663-0595,19879.753576
78,N6f2wTV9Gd7iqFe2norBAg,7-eleven-bakersfield-19,7-Eleven,https://s3-media2.fl.yelpcdn.com/bphoto/WgKK4Y...,False,https://www.yelp.com/biz/7-eleven-bakersfield-...,6,"[{'alias': 'convenience', 'title': 'Convenienc...",2.5,"{'latitude': 35.3537992496193, 'longitude': -1...","[pickup, delivery]",$,"{'address1': '5301 Stockdale Hwy', 'address2':...",+16618324455,(661) 832-4455,20579.402904
79,A5AhA_LYfGMmpWPVhVkyBQ,trader-joes-bakersfield,Trader Joe's,https://s3-media4.fl.yelpcdn.com/bphoto/Ld2H_y...,False,https://www.yelp.com/biz/trader-joes-bakersfie...,170,"[{'alias': 'grocery', 'title': 'Grocery'}]",4.5,"{'latitude': 35.357022, 'longitude': -119.093527}",[],$$,"{'address1': '8200 Stockdale Hwy', 'address2':...",+16618378863,(661) 837-8863,21348.660267
80,WOkEQhQv2ZRLRC-Z15iLIg,7-eleven-bakersfield-18,7-Eleven,https://s3-media4.fl.yelpcdn.com/bphoto/9Sm52P...,False,https://www.yelp.com/biz/7-eleven-bakersfield-...,4,"[{'alias': 'convenience', 'title': 'Convenienc...",2.0,"{'latitude': 35.3972717, 'longitude': -118.985...","[delivery, pickup]",$,"{'address1': '2636 River Blvd', 'address2': ''...",+16613222995,(661) 322-2995,25935.471331


In [20]:
# checking for duplicates
final_df.duplicated(subset="id").sum()

0

In [22]:
final_df.to_csv("Data/final_results_sushi.csv.gz", compression="gzip", index=False)